Intro bla bla

In [2]:
# importing libraries
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import polars as pl
import matplotlib.pyplot as plt
import pyarrow.parquet as pq
import seaborn as sns
import missingno as msno
from pandas.api.types import is_datetime64_ns_dtype
import gc  # garbage collector module

import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
import plotly.offline
import pyarrow.parquet as pq

import kaleido
import os
from IPython.display import Image, display

import warnings
warnings.filterwarnings("ignore")  # ignore any warnings in the code execution

ModuleNotFoundError: No module named 'polars'

In [ ]:
!pip install polars

In [4]:
# define the common path
# change this to your local path
path = 'gs://sleep-nov2023/data/'

In [5]:
# reading a Parquet file
# train_series_aggregated = pd.read_parquet(path + 'full_train_and_split/train_series_aggregated.parquet')
# X_test = pd.read_parquet(path + 'full_train_and_split/X_test.parquet')
X_train = pd.read_parquet(path + 'full_train_and_split/X_train.parquet')
# y_test = pd.read_parquet(path + 'full_train_and_split/y_test.parquet')
y_train = pd.read_parquet(path + 'full_train_and_split/y_train.parquet')

In [6]:
X_train.head()

,timestamp,anglez,enmo,step,series_id
0,2018-08-14 15:30:00,3.084700,0.0229,0.0,038441c925bb
1,2018-08-14 15:31:00,68.460503,0.0395,12.0,038441c925bb
2,2018-08-14 15:32:00,-79.998703,0.0691,24.0,038441c925bb
3,2018-08-14 15:33:00,-80.008202,0.0140,36.0,038441c925bb
4,2018-08-14 15:34:00,-80.025299,0.0141,48.0,038441c925bb


In [7]:
y_train.head()

,series_id,night,event,step,timestamp,year,month,day,hour
0,038441c925bb,1,onset,4992.0,2018-08-14 22:26:00,2018.0,8.0,14.0,22.0
1,038441c925bb,1,wakeup,10932.0,2018-08-15 06:41:00,2018.0,8.0,15.0,6.0
2,038441c925bb,2,onset,20244.0,2018-08-15 19:37:00,2018.0,8.0,15.0,19.0
3,038441c925bb,2,wakeup,27492.0,2018-08-16 05:41:00,2018.0,8.0,16.0,5.0
4,038441c925bb,3,onset,39996.0,2018-08-16 23:03:00,2018.0,8.0,16.0,23.0


In [8]:
# normalise data
# use standard scaler for `enmo`
# use deviation by 90 for `anglez` because it is a 90 degree angle

from sklearn.preprocessing import StandardScaler
X_train['enmo'] = StandardScaler().fit_transform(X_train[['enmo']])
X_train['anglez'] = X_train['anglez'] / 90

#### Calculate the locomotor Inactivity During Sleep (LIDS) out of the ENMO signal

LIDS refers to "Locomotor Inactivity During Sleep", a concept derived from the non-linear conversion of locomotor activity. This conversion helps reveal movement patterns that correspond directly to ultradian sleep cycles, providing a means to replicate laboratory sleep parameters dynamically.

Based on two different publications <sup>1, 2</sup>, we calculated `lids` = 100/ (activity count + 1), where activity count is computed using a 10-minute moving sum over max(0, `enmo_clean` − 0.02). LIDS is then smoothed using a moving average over a 30-min window.

<sup>1</sup>
<a
    href=https://doi.org/10.1016/j.cub.2017.11.063>
    Dynamics and Ultradian Structure of Human Sleep in Real Life
</a>

<sup>2</sup>
<a
    href=https://doi.org/10.1038/s41598-020-79217-x>
    Sleep classification from wrist-worn accelerometer data using random forests
</a>


In [9]:
# calculate ENMO - 0.02 and take max with 0
activity_count = np.maximum(0, X_train['enmo'] - 0.02)

# compute 10-minute moving sum
activity_count_10min_sum = activity_count.rolling(window=10).sum()

# calculate LIDS
lids = 100 / (activity_count_10min_sum + 1)

# add lids to the original sleep dataframe
X_train['lids'] = lids

In [10]:
X_train.head()

,timestamp,anglez,enmo,step,series_id,lids
0,2018-08-14 15:30:00,0.034274,-0.358965,0.0,038441c925bb,NaN
1,2018-08-14 15:31:00,0.760672,-0.285397,12.0,038441c925bb,NaN
2,2018-08-14 15:32:00,-0.888874,-0.154216,24.0,038441c925bb,NaN
3,2018-08-14 15:33:00,-0.888980,-0.398408,36.0,038441c925bb,NaN
4,2018-08-14 15:34:00,-0.889170,-0.397965,48.0,038441c925bb,NaN


In [11]:
X_train.shape

(7778985, 6)

#### Calculate additional values like entropy and peak of amplitude during sleep duration
Based on one publication <sup>1</sup>, we incorporated additional signal metrics that can be calculated.

| Feature                   | Description                                           |
|---------------------------|-------------------------------------------------------|
| RollingMean_{window_size}min (anglez) | Rolling mean for a specified time window in 'anglez' |
| RollingStd_{window_size}min (anglez)  | Rolling standard deviation for a specified time window in 'anglez' |
| RollingMin_{window_size}min (anglez)  | Rolling minimum for a specified time window in 'anglez' |
| RollingMax_{window_size}min (anglez)  | Rolling maximum for a specified time window in 'anglez' |
| RollingMedian_{window_size}min (anglez) | Rolling median for a specified time window in 'anglez' |
| Entropy_{column_name}_{resolution} (anglez) | Entropy for 'anglez' at specified resolution |
| RollingMean_{window_size}min (enmo) | Rolling mean for a specified time window in 'enmo' |
| RollingStd_{window_size}min (enmo)  | Rolling standard deviation for a specified time window in 'enmo' |
| RollingMin_{window_size}min (enmo)  | Rolling minimum for a specified time window in 'enmo' |
| RollingMax_{window_size}min (enmo)  | Rolling maximum for a specified time window in 'enmo' |
| RollingMedian_{window_size}min (enmo) | Rolling median for a specified time window in 'enmo' |
| Entropy_{column_name}_{resolution} (enmo) | Entropy for 'enmo' at specified resolution |
| RollingMean_{window_size}min (lids) | Rolling mean for a specified time window in 'lids' |
| RollingStd_{window_size}min (lids)  | Rolling standard deviation for a specified time window in 'lids' |
| RollingMin_{window_size}min (lids)  | Rolling minimum for a specified time window in 'lids' |
| RollingMax_{window_size}min (lids)  | Rolling maximum for a specified time window in 'lids' |
| RollingMedian_{window_size}min (lids) | Rolling median for a specified time window in 'lids' |
| Entropy_{column_name}_{resolution} (lids) | Entropy for 'lids' at specified resolution |

Window size: 1 to 30 minutes (5-minute steps).
Resolution: 'high' (bin size = 200) or 'low' (bin size = 20).

<sup>1</sup>
<a
    href=https://doi.org/10.1038/s41598-020-79217-x>
    Sleep classification from wrist-worn accelerometer data using random forests
</a>

In [12]:
# define a function to calculate the rolling mean, std, min, max, and median
# for different time windows from 1 minute to 30 minutes

def calculate_stat_features(signal):
    """
    Calculate rolling statistics for different time windows.

    Parameters:
    - signal: pd.Series, input time-series data

    Returns:
    - pd.DataFrame, containing rolling mean, std, min, max, median
    """

    # Create a DataFrame to store the results
    result_df = pd.DataFrame(index=signal.index)

    # Define the list of window sizes (1 minute to 30 minutes with 5-minute steps)
    window_sizes = range(1, 31, 5)

    # Calculate rolling statistics for each window size
    # By using .shift(1), you are effectively moving the values one step forward in time, 
    # which helps in aligning the rolling statistics with the original time series data, and it 
    # avoids NaN values in the first row.
    for window_size in window_sizes:
        rolling_window = signal.rolling(window=window_size)
        result_df[f'RollingMean_{window_size}min'] = rolling_window.mean().shift(1)
        result_df[f'RollingStd_{window_size}min'] = rolling_window.std().shift(1)
        result_df[f'RollingMin_{window_size}min'] = rolling_window.min().shift(1)
        result_df[f'RollingMax_{window_size}min'] = rolling_window.max().shift(1)
        result_df[f'RollingMedian_{window_size}min'] = rolling_window.median().shift(1)

    return result_df

In [13]:
# define a function to calculate the entropy of a signal
from scipy.stats import entropy

def calculate_entropy_features(signal, column_name, resolution='high', bin_size=None):
    """
    Calculate the entropy of a signal and return it as a DataFrame.

    Parameters:
    - signal: pd.Series or float, input signal
    - column_name: str, name of the column for creating feature names
    - resolution: str, 'high' or 'low'
    - bin_size: int, number of bins for 'low' resolution entropy (default is 20, ignored for 'high' resolution)

    Returns:
    - pd.Series or pd.DataFrame, containing entropy values
    """

    entropy_values = pd.Series()

    if pd.Series(signal).isna().all():
        # Handle case where all values are NaN
        entropy_values[f'Entropy_{column_name}_{resolution}'] = np.nan
    elif isinstance(signal, float) or pd.Series(signal).isna().any():
        # Handle case where signal is a single value or contains NaN values
        entropy_values[f'Entropy_{column_name}_{resolution}'] = 0.0
    else:
        # Convert the signal to a numpy array if it's a pandas Series
        signal_array = np.array(signal)

        # Check if the resolution is 'low' or 'high'
        if resolution == 'high':
            # Use binning for high-resolution entropy
            probabilities = np.histogram(signal_array[~np.isnan(signal_array)], bins=200, density=True)[0]
            entropy_values[f'Entropy_{column_name}_{resolution}'] = entropy(probabilities, base=2)

        elif resolution == 'low':
            # Use binning for low-resolution entropy
            if bin_size is None:
                bin_size = 20  # Default bin size for 'low' resolution

            # Calculate the number of bins based on the specified bin size
            bins = np.linspace(min(signal_array), max(signal_array), bin_size + 1)

            # Use histogram to count occurrences in each bin
            counts, _ = np.histogram(signal_array[~np.isnan(signal_array)], bins=bins)
            probabilities = counts / len(signal_array[~np.isnan(signal_array)])
            entropy_values[f'Entropy_{column_name}_{resolution}'] = entropy(probabilities, base=2)

        else:
            raise ValueError("Invalid resolution. Use 'low' or 'high'.")

    return entropy_values



In [14]:
# Define the columns to process
columns_to_process = ['enmo', 'anglez', 'lids']

# Results storage
all_stats = []
all_entropy_high = []
all_entropy_low = []

# Set the chunk size based on your available memory
chunk_size = 1000000

unique_series_ids = X_train['series_id'].unique()

In [ ]:
# Iterate through each column
for column in columns_to_process:
    # Initialize lists to store results for each chunk
    chunk_stats = []
    chunk_entropy_high = []
    chunk_entropy_low = []

    # Iterate over chunks of the dataset
    for start in range(0, len(X_train), chunk_size):
        end = start + chunk_size
        chunk_data = X_train.iloc[start:end]

        # Calculate statistics for the current chunk
        stats = chunk_data.groupby(['series_id', 'step'])[column].apply(calculate_stat_features)
        stats.columns = [f'{column}_{col}' for col in stats.columns]
        chunk_stats.append(stats)

        # Calculate entropy for high resolution for the current chunk
        entropy_high = chunk_data.groupby(['series_id', 'step'])[column].apply(
            lambda x: calculate_entropy_features(x, column_name=column, resolution='high')
        )
        entropy_high.index = [f'{column}_{index}' for index in entropy_high.index]
        chunk_entropy_high.append(entropy_high)

        # Calculate entropy for low resolution for the current chunk
        entropy_low = chunk_data.groupby(['series_id', 'step'])[column].apply(
            lambda x: calculate_entropy_features(x, column_name=column, resolution='low', bin_size=20)
        )
        entropy_low.index = [f'{column}_{index}' for index in entropy_low.index]
        chunk_entropy_low.append(entropy_low)

    # Concatenate the results for each column and append to the final lists
    all_stats.append(pd.concat(chunk_stats, keys=unique_series_ids))
    all_entropy_high.append(pd.concat(chunk_entropy_high, keys=unique_series_ids))
    all_entropy_low.append(pd.concat(chunk_entropy_low, keys=unique_series_ids))

In [ ]:
# Concatenate the results for each column and append to the final lists
all_stats_df = pd.concat(all_stats, keys=unique_series_ids)
all_entropy_high_df = pd.concat(all_entropy_high, keys=unique_series_ids)
all_entropy_low_df = pd.concat(all_entropy_low, keys=unique_series_ids)

# Concatenate the final results for all series_ids
X_train = pd.concat([X_train, all_stats_df, all_entropy_high_df, all_entropy_low_df], axis=1)


In [ ]:
# saving the modified X_train to the data folder
X_train.to_parquet(path + 'full_train_and_split/X_train_modified.parquet')


### Feature Engineering Summary

1. **Data Normalisation:**
   - The 'enmo' column is normalised using the Standard Scaler.
   - The 'anglez' column is scaled by dividing it by 90.

2. **Calculation of Locomotor Inactivity During Sleep (LIDS):**
   - LIDS is calculated based on the formula: `lids = 100 / (activity count + 1)`, where activity count is computed using a 10-minute moving sum over max(0, `enmo_clean` − 0.02).
   - The activity count is derived as the maximum of 0 and the difference between 'enmo' and 0.02.
   - LIDS is smoothed using a 30-minute moving average.

3. **Rolling Statistics Calculation:**
   - A function `calculate_stat_features` is defined to calculate rolling mean, standard deviation, minimum, maximum, and median for different time windows (1 minute to 30 minutes with 5-minute steps).
   - The functions are applied to the 'enmo', 'anglez', and 'lids' columns, resulting in statistical features for each.

4. **Entropy Calculation:**
   - A function `calculate_entropy_features` is defined to calculate entropy of a signal.
   - Entropy is calculated at both high and low resolutions (20 bins for low resolution).
   - The calculated entropy features are added to the dataset for 'enmo', 'anglez', and 'lids' columns.


In [ ]:
X_train.groupby('series_id').describe().T

In [ ]:
X_train.groupby('series_id').apply(lambda x: x.isnull().sum())

In [ ]:
# Calculate the percentage of NaN values within each series_id
nan_percentage = X_train.groupby('series_id').apply(lambda x: (x.isnull().sum() / len(x)) * 100)

# Display the result
print(nan_percentage)

In [ ]:
# Set the threshold for missing data percentage
threshold = 30

# Calculate the percentage of NaN values within each series_id
nan_percentage = X_train.groupby('series_id').apply(lambda x: (x.isnull().sum() / len(x)) * 100)

# Filter features with >30% missing data within one series_id
features_with_high_nan = nan_percentage[nan_percentage > threshold]

# Display the result
features_with_high_nan


In [ ]:
X_train.head(20)